In [1]:
from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from astropy.wcs import WCS
# specutils packages
from specutils import Spectrum1D
from specutils.analysis import line_flux
from specutils.fitting import fit_generic_continuum
from specutils import SpectralRegion
from specutils.analysis import equivalent_width
from specutils.analysis import centroid
from specutils.analysis import moment
from specutils.manipulation import extract_region
from specutils.fitting import find_lines_threshold
from specutils.analysis import gaussian_sigma_width, gaussian_fwhm, fwhm, fwzi
import warnings
with warnings.catch_warnings():  # Ignore warnings
    warnings.simplefilter('ignore')
quantity_support()

<astropy.visualization.units.quantity_support.<locals>.MplQuantityConverter at 0x7f165a70d580>

In [2]:
# OUR PN
hdu = fits.open("../Spectra-lamostdr7/spec-56581-VB031N50V1_sp08-218.fits")
hdudata = hdu[0].data
wl = hdudata[2]
Flux = hdudata[0]

In [3]:
# Defining units astropy
rel_flux = u.def_unit('Relative~flux')
rel_flux.decompose()

Unit("Relative~flux")

In [4]:
lamb = wl * u.AA 
flux = Flux * rel_flux
spec = Spectrum1D(spectral_axis=lamb, flux=flux) 

In [5]:
def closest(lst, K):
    '''find the closest number'''
    lst = np.array(lst)
    idx = (np.abs(lst - K)).argmin()
    return lst[idx]

def find_line(wl_vacuum, spec):
    '''
    This function find the line
    in a wavelenght range
    '''
    line_region = SpectralRegion((wl_vacuum-15)*u.AA, (wl_vacuum+15)*u.AA)
    line_spec = extract_region(spec, line_region)
    with warnings.catch_warnings():  # Ignore warnings
        warnings.simplefilter('ignore')
        line_spec = find_lines_threshold(line_spec, noise_factor = 3) # find the lines
        print("Number of line found:", len(line_spec))
    if len(line_spec) > 1:
        wl_line = closest(line_spec['line_center'].value, wl_vacuum)
        mask = line_spec['line_center'].value == wl_line
        line_spec_mask = line_spec[mask]
    else:
        line_spec_mask = line_spec
    return line_spec_mask

In [6]:
from specutils.analysis import centroid
def cal_fwhm(wl_vacuum, spec):
    line_spec_mask = find_line(wl_vacuum,  spec)
    line_region_ = SpectralRegion(line_spec_mask['line_center'] - 5 * u.AA, 
                                  line_spec_mask['line_center'] + 5 * u.AA)
    centr = centroid(spec, line_region_)
    line_region_centr = SpectralRegion(centr - 5 * u.AA, centr + 5 * u.AA)
    sub_spectrum_line = extract_region(spec, line_region_centr)
    fwhm_ = gaussian_fwhm(sub_spectrum_line)
    return fwhm_

In [7]:
lines = {"[NeIII]+H7": 3967.470,
         "Hdelta": 4101.742,
         "Hgamma": 4340.471,
         "HeII": 4685.99,
         "Hbeta": 4861.333,
         "[OIII]4958": 4958.911,
         "[OIII]5006": 5006.843,
         "[FeIII]": 5412.12,
         "Halpha": 6564.614,
         }

In [8]:
#Estimating the flux line for the models
v_ = []
fw = []
c = 3.e5
for vv, tt in lines.items():
    fwhmm = cal_fwhm(tt, spec)
    v = (c * fwhmm.value) / tt
    fw.append(fwhmm)
    v_.append(v)
    

Number of line found: 7
Number of line found: 4
Number of line found: 3
Number of line found: 1
Number of line found: 1
Number of line found: 1
Number of line found: 1
Number of line found: 1
Number of line found: 1


In [9]:
print(v_, fw)

[392.80983459119346, 292.7831925725172, 257.2463180573366, 220.2023049238924, 219.0074588148655, 237.7750868238413, 201.71564171619613, 259.79165760766637, 195.81415694728292] [<Quantity 5.19487078e-13 Angstrom>, <Quantity 4.00307039e-13 Angstrom>, <Quantity 3.72190061e-13 Angstrom>, <Quantity 3.43955266e-13 Angstrom>, <Quantity 3.54889396e-13 Angstrom>, <Quantity 3.93035165e-13 Angstrom>, <Quantity 3.3665285e-13 Angstrom>, <Quantity 4.68674542e-13 Angstrom>, <Quantity 4.28481452e-13 Angstrom>]


In [10]:
np.mean(v_)

253.01618356164352